<a href="https://colab.research.google.com/github/sanjivch/100-days-of-gpu/blob/main/CUDA__Day03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             !python --version
!nvcc --version
!pip install nvcc4jupyter

Python 3.11.11
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp_a14u0td".


In [6]:
%%cuda
#include <iostream>

__global__ void vecMatMul(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < N) { // guard block
      float sum=0.0f;
      for (int j = 0; j < N; j++) {
         sum += A[i*N + j]*B[j];
      }
      C[i]=sum;
}}

int main() {
    //initialize the matrix
    const int N = 10;
    float *A, *B, *C;

    // Initialize matrices/ vector
    A = (float *)malloc( N*N* sizeof(float));
    B = (float *)malloc(N*sizeof(float));
    C = (float *)malloc(N*sizeof(float));

    // Assign variables
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            A[i * N + j] = 1.0f;
        }
        B[i] = 2.0f;
        C[i] = 0.0f;
    }

    float *d_a, *d_b,*d_c;

    //Allocate memory
    cudaMalloc(&d_a, N * N * sizeof(float));
    cudaMalloc(&d_b, N * sizeof(float));
    cudaMalloc(&d_c, N * sizeof(float));

    // Move to device
    cudaMemcpy(d_a, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, B, N * sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel - Define blocksize and gridsize
    int blocksize=256;
    int gridsize = (N + blocksize - 1) / blocksize;
    vecMatMul<<<gridsize,blocksize>>>(d_a, d_b, d_c, N);

    cudaDeviceSynchronize();
    cudaMemcpy(C,d_c,N*sizeof(float),cudaMemcpyDeviceToHost);

    printf("A, B, C:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {

            printf("%.2f ", A[i * N + j]);
            printf("%.2f ", B[i ]);
             printf("%.2f ", C[i ]);
        }
        printf("\n");
    }




    // Free resources
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

}

A, B, C:
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 
1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 0.00 1.00 2.00 